<a href="https://colab.research.google.com/github/TCipresso/Chess/blob/main/ChessAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Chess AI

this is the AI I made for my AI class.

Uses MiniMax algorithm mixed with PST's to understadn what is the best place ont he board. Cosntantly checks the current baord to get the most recent results.

Instructions on how to use google colab:
Either copy paste the entire code or run this in colab by pressing the play button on each block of code. I gave 2 main blocks:

1 For showing my AI VS a Random move AI this is great for speed.

2 For shwoing my AI VS you the user.

Thank you for reading this have a great day!

In [1]:
# your code goes here
import string
import random
import os
import sys
import time
from IPython.display import clear_output


In [2]:


ChessBoard = []
def ChessBoardSetup():
    # initialize and return a chess board - create a 2D 8x8 array that has the value for each cell
    # USE the following characters for the chess pieces - lower-case for BLACK and upper-case for WHITE
    # . for empty board cell
    # p/P for pawn
    # r/R for rook
    # t/T for knight
    # b/B for bishop
    # q/Q for queen
    # k/K for king
    global ChessBoard
    # initialize and return a chess board - create a 2D 8x8 array that has the value for each cell
    ChessBoard = [
        ['r', 't', 'b', 'q', 'k', 'b', 't', 'r'],
        ['p', 'p', 'p', 'p', 'p', 'p', 'p', 'p'],
        ['.', '.', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.', '.'],
        ['P', 'P', 'P', 'P', 'P', 'P', 'P', 'P'],
        ['R', 'T', 'B', 'Q', 'K', 'B', 'T', 'R']
    ]
    return ChessBoard


def DrawBoard(BoardDraw):
    # write code to print the board - following is one print example
    # r t b q k b t r
    # p p p p p p p p
    # . . . . . . . .
    # . . . . . . . .
    # . . . . . . . .
    # . . . . . . . .
    # P P P P P P P P
    # R T B Q K B T R
    # Print the board
    for Row in BoardDraw:
        print(' '.join(Row))


def MovePiece(BoardMove, MainRow, MainCol, TooRow, TooCol):
    # write code to move the one chess piece
    # you do not have to worry about the validity of the move - this will be done before calling this function
    # this function will at least take the move (from-piece and to-piece) as input and return the new board layout
   # Moves a piece from the provided source to the destination
    # The validity of the move should be checked before calling this function
    TargetPiece = BoardMove[MainRow][MainCol]
    BoardMove[TooRow][TooCol] = TargetPiece
    BoardMove[MainRow][MainCol] = '.'
    return BoardMove

ChessBoard = ChessBoardSetup()

print("\nMoving piece on A2 to A4\n")
CurrBoard = MovePiece(ChessBoard, 6, 0, 4, 0)
DrawBoard(CurrBoard)




Moving piece on A2 to A4

r t b q k b t r
p p p p p p p p
. . . . . . . .
. . . . . . . .
P . . . . . . .
. . . . . . . .
. P P P P P P P
R T B Q K B T R


In [3]:
# return True if the input move (from-square and to-square) is legal, else False
# this is the KEY function which contains the rules for each piece type

def IsMoveLegal(chessboard, FromRow, FromCol, ToRow, ToCol):
    FromPiece = chessboard[FromRow][FromCol]
    ToPiece = chessboard[ToRow][ToCol]

    # if from-square is the same as to-square
    if FromRow == ToRow and FromCol == ToCol:
        return False

    # if the from-piece is a "pawn"
    if FromPiece.lower() == 'p':
        PieceDirection = 1 if FromPiece == 'p' else -1  # Black pawn moves down, white moves up
        if ToPiece == '.' and FromCol == ToCol:
            # case - pawn wants to move one step forward (or backward if white)
            if ToRow - FromRow == PieceDirection:
                return True
            # case - pawn can move two spaces forward (or backward if white) ONLY if pawn on starting row
            elif (FromRow in [1, 6] and abs(ToRow - FromRow) == 2 and IsClearPath(chessboard, FromRow, FromCol, ToRow, ToCol)):
                return True
        # case - pawn attacks the enemy piece if diagonal
        elif abs(ToCol - FromCol) == 1 and ToRow - FromRow == PieceDirection and ToPiece != '.' and ToPiece.isupper() != FromPiece.isupper():
          return True

    # else if the from-piece is a "rook"
    elif FromPiece.lower() == 'r':
    # if to-square is either in the same row or column as the from-square
        if ToRow == FromRow or ToCol == FromCol:
        # if to-square is either empty or contains a piece that belongs to the enemy team
            if ToPiece == '.' or ToPiece.isupper() != FromPiece.isupper():
            # if IsClearPath() - a clear path exists between from-square and to-square
                if IsClearPath(chessboard, FromRow, FromCol, ToRow, ToCol):
                    return True


    # else if the from-piece is a "bishop"
    elif FromPiece.lower() == 'b':

    # if to-square is diagonal wrt from-square
        if abs(ToRow - FromRow) == abs(ToCol - FromCol):
        # if to-square is either empty or contains a piece that belongs to the enemy team
            if ToPiece == '.' or ToPiece.isupper() != FromPiece.isupper():
            # if IsClearPath() - a clear path exists between from-square and to-square
                if IsClearPath(chessboard, FromRow, FromCol, ToRow, ToCol):
                    return True


    # else if the from-piece is a "queen"
    elif FromPiece.lower() == 'q':
        # rook logic
        if (ToRow == FromRow or ToCol == FromCol) and (ToPiece == '.' or ToPiece.isupper() != FromPiece.isupper()) and IsClearPath(chessboard, FromRow, FromCol, ToRow, ToCol):
            return True
        # bishop logic
        elif abs(ToRow - FromRow) == abs(ToCol - FromCol) and (ToPiece == '.' or ToPiece.isupper() != FromPiece.isupper()) and IsClearPath(chessboard, FromRow, FromCol, ToRow, ToCol):
            return True

    # else if the from-piece is a "knight"
    elif FromPiece.lower() == 't':
      # calculate the col-diff = to-square-col - from-square-col
        ColDiff = ToCol - FromCol
        # calculate the row-diff = to-square-row - from-square-row
        RowDiff = ToRow - FromRow
        # if to-square is either empty or contains a piece that belongs to the enemy team
        if ToPiece == '.' or ToPiece.isupper() != FromPiece.isupper():
          # return True for any of the following cases:
            KnightMoves = [(1,-2), (2,-1), (2,1), (1,2), (-1,-2), (-2,-1), (-2,1), (-1,2)]
            if (ColDiff, RowDiff) in KnightMoves:
                return True

    # else if the from-piece is a "king"
    elif FromPiece.lower() == 'k':
        ColDiff = ToCol - FromCol
        RowDiff = ToRow - FromRow
        if ToPiece == '.' or ToPiece.isupper() != FromPiece.isupper():
            KingMoves = [(0,1), (1,0), (0,-1), (-1,0), (1,1), (-1,-1), (1,-1), (-1,1)]
            if (ColDiff, RowDiff) in KingMoves:
                return True

    return False  # if none of the other True's are hit above


# Let's test the function with a few examples:
def GetListOfLegalMoves(chessboard2, Currentplayer2, FromRow2, FromCol2):
    # Initialize the list of legal moves, i.e., to-square locations to []
    LegalMoves = []

    # Go through all squares on the board
    for ToRow2 in range(8):
        for ToCol2 in range(8):
            # Call IsMoveLegal() with input as from-square and to-square and save the returned value
            IsLegalMove = IsMoveLegal(chessboard2, FromRow2, FromCol2, ToRow2, ToCol2)

            # Check if the move is capturing a king, if so, skip this move
            if chessboard2[ToRow2][ToCol2] in ['K', 'k']:
                continue

            # If returned value is True
            if IsLegalMove:
                # Call DoesMovePutPlayerInCheck() with input as from-square and to-square and save the returned value
                IsCheck = DoesMovePutPlayerInCheck(chessboard2, FromRow2, FromCol2, ToRow2, ToCol2, Currentplayer2)

                # If returned value is False
                if IsCheck == False:
                    # Append this move (to-square) as a legal move
                    LegalMoves.append((ToRow2, ToCol2))

    # Return the list of legal moves, i.e., to-square locations
    return LegalMoves




# gets a list of all pieces for the current player that have legal moves
def GetPiecesWithLegalMoves(chessboard3, CurrentPlayer3):
    # Initialize the list of pieces with legal moves to []
    PiecesWithLegalMoves = []

    # Go through all squares on the chessboard
    for Row3 in range(8):
        for Col3 in range(8):
            # If the square contains a Piece that belongs to the current CurrentPlayer's team
            Piece3 = chessboard3[Row3][Col3]
            if (CurrentPlayer3 == 'white' and Piece3.isupper()) or (CurrentPlayer3 == 'black' and Piece3.islower()):
                # Call GetListOfLegalMoves() to get a list of all legal moves for the selected Piece/square
                LegalMoves = GetListOfLegalMoves(chessboard3, CurrentPlayer3, Row3, Col3)

                # If there are any legal moves
                if len(LegalMoves) > 0:
                    # Append this Piece to the list of PiecesWithLegalMoves
                    PiecesWithLegalMoves.append((Row3, Col3))

    # Return the final list of PiecesWithLegalMoves
    return PiecesWithLegalMoves




# returns True if the current player is in checkmate, else False
#this one gave me a lto of trouble so I had to make it make a copy and refernce taht copy of the board to find any possible moevs if ther is none return true(checkmate)
def IsCheckmate(chessboard4, CurrentPlayer4):
    for Row4 in range(8):
        for Col4 in range(8):
            # Only consider pieces of the current player
            Piece4 = chessboard4[Row4][Col4]
            if Piece4 != '.' and (Piece4.isupper() == (CurrentPlayer4 == 'white')):
                LegalMoves = GetListOfLegalMoves(chessboard4, CurrentPlayer4, Row4, Col4)
                for Move4 in LegalMoves:
                    # Make a copy of the board
                    CopyBoard4 = [row[:] for row in chessboard4]
                    # try the move on the copied board
                    CopyBoard4[Move4[0]][Move4[1]] = Piece4
                    CopyBoard4[Row4][Col4] = '.'
                    # If this move results in the player not being in check, it means it's not checkmate
                    if not IsInCheck(CopyBoard4, CurrentPlayer4):
                        return False
    return True  # if no legal moves found checkamte





# returns True if the given player is in Check state
def IsInCheck(chessboard5, CurrentPlayer5):
    # Find given player's King's location
    KingLocation = None
    for Row5 in range(8):
        for Col5 in range(8):
            if chessboard5[Row5][Col5] == 'K' and CurrentPlayer5 == 'white':
                KingLocation = (Row5, Col5)
            elif chessboard5[Row5][Col5] == 'k' and CurrentPlayer5 == 'black':
                KingLocation = (Row5, Col5)

    # Check if king was found; if not, return False
    if KingLocation is None:
        return False

    KingRow, KingCol = KingLocation

    # Go through all squares on the board
    for Row5 in range(8):
        for Col5 in range(8):
            Piece5 = chessboard5[Row5][Col5]

            # Check if there is a Piece at that location and that Piece is an enemy
            if Piece5 != '.' and ((CurrentPlayer5 == 'white' and Piece5.islower()) or (CurrentPlayer5 == 'black' and Piece5.isupper())):

                # Call IsMoveLegal() for the enemy player from that square to the KingLocation
                if IsMoveLegal(chessboard5, Row5, Col5, KingRow, KingCol):
                    return True  # If the value returned is True, the king is in check

    # Return False at the end if king is not in check
    return False



# helper function to figure out if a move is legal for straight-line moves (rooks, bishops, queens, pawns)
# returns True if the path is clear for a move (from-square and to-square), non-inclusive
def IsClearPath(chessboard6, FromRow6, FromCol6, ToRow6, ToCol6):
    # if the from and to squares are only one square apart
    if abs(FromRow6 - ToRow6) <= 1 and abs(FromCol6 - ToCol6) <= 1:
        return True
    # else
    else:
        # if to-square is in the +ve vertical direction from from-square
        if ToRow6 > FromRow6 and FromCol6 == ToCol6:
            NewFromRow, NewFromCol = FromRow6 + 1, FromCol6
        # else if to-square is in the -ve vertical direction from from-square
        elif ToRow6 < FromRow6 and FromCol6 == ToCol6:
            NewFromRow, NewFromCol = FromRow6 - 1, FromCol6
        # else if to-square is in the +ve horizontal direction from from-square
        elif ToCol6 > FromCol6 and FromRow6 == ToRow6:
            NewFromRow, NewFromCol = FromRow6, FromCol6 + 1
        # else if to-square is in the -ve horizontal direction from from-square
        elif ToCol6 < FromCol6 and FromRow6 == ToRow6:
            NewFromRow, NewFromCol = FromRow6, FromCol6 - 1
        # else if to-square is in the SE diagonal direction from from-square
        elif ToRow6 > FromRow6 and ToCol6 > FromCol6:
            NewFromRow, NewFromCol = FromRow6 + 1, FromCol6 + 1
        # else if to-square is in the SW diagonal direction from from-square
        elif ToRow6 > FromRow6 and ToCol6 < FromCol6:
            NewFromRow, NewFromCol = FromRow6 + 1, FromCol6 - 1
        # else if to-square is in the NE diagonal direction from from-square
        elif ToRow6 < FromRow6 and ToCol6 > FromCol6:
            NewFromRow, NewFromCol = FromRow6 - 1, FromCol6 + 1
        # else if to-square is in the NW diagonal direction from from-square
        elif ToRow6 < FromRow6 and ToCol6 < FromCol6:
            NewFromRow, NewFromCol = FromRow6 - 1, FromCol6 - 1
        else:
            return False

        # if new-from-square is not empty
        if chessboard6[NewFromRow][NewFromCol] != '.':
            return False
        # else
        else:
            # return the result from the recursive call of IsClearPath() with the new-from-square and to-square
            return IsClearPath(chessboard6, NewFromRow, NewFromCol, ToRow6, ToCol6)


# makes a hypothetical move (from-square and to-square)
# returns True if it puts current player into check
def DoesMovePutPlayerInCheck(chessboard7, FromRow7, FromCol7, ToRow7, ToCol7, CurrentPlayer7):
    #given the move (from-square and to-square), find the 'from-piece' and 'to-piece'
    FromPiece7 = chessboard7[FromRow7][FromCol7]
    ToPiece7 = chessboard7[ToRow7][ToCol7]
    #make the move temporarily by changing the baord
    chessboard7[ToRow7][ToCol7] = FromPiece7
    chessboard7[FromRow7][FromCol7] = '.'
    # Check if the move puts the player in check.
    IsCheckk = IsInCheck(chessboard7, CurrentPlayer7)
    # Undo the temporary move.
    chessboard7[FromRow7][FromCol7] = FromPiece7
    chessboard7[ToRow7][ToCol7] = ToPiece7

    return IsCheckk




In [4]:
#this handles input from the player to enter what pieces tehy wnat to move and to where.
#in order for any of my test functions to work need to run this as a must first.
def GetInput():
    while True:
        try:
            FromSquareTEST, ToSquareTest = input("Please enter the square of the piece you want to move and the square you want to move the piece to (EX: enter: (a2 a4) to move a2 to a4'): ").split()
            FromRowTEST, FromColTEST = CoordConverter(FromSquareTEST)
            ToRowTEST, ToColTEST = CoordConverter(ToSquareTest)

            if FromRowTEST in range(8) and FromColTEST in range(8) and ToRowTEST in range(8) and ToColTEST in range(8):
                return FromSquareTEST, ToSquareTest
            else:
                raise ValueError
        except ValueError:
            print("Invalid input try again.")

#allows user to enter say a2 to a4 and program uindertadns it.
def CoordConverter(Coord):
    col = ord(Coord[0].upper()) - ord('A')
    row = 8 - int(Coord[1])
    return row, col

In [6]:

def GetRandomMove(randBoard, CurrentPlayerRand):
    # pick a random piece and a random legal move for that piece
    # Get all the CurrentPlayerRand's pieces on the board
    PiecesPos = [(RowRand, ColRand) for RowRand in range(8) for ColRand in range(8)
if (randBoard[RowRand][ColRand].isupper() and CurrentPlayerRand == "white") or (randBoard[RowRand][ColRand].islower() and CurrentPlayerRand == "black")]
    random.shuffle(PiecesPos)  # Shuffle the pieces for randomness
    for RowRand, ColRand in PiecesPos:
        LegalMovesRand = []  # Store all legal moves  for random current piecwe
        for i in range(8):
            for j in range(8):
                if IsMoveLegal(randBoard, RowRand, ColRand, i, j):
                    LegalMovesRand.append((i, j))
        # If the current piece has any legal moves, select one randomly
        if LegalMovesRand:
            ToRowRand, ToColRand = random.choice(LegalMovesRand)
            return (RowRand, ColRand), (ToRowRand, ToColRand)
    return None


In [8]:
# NEED TO RUN THIS BLOCK FOR MINMAX AI. these are piece-square-tables for the next section. My AI was not very intelligent so I did some reaserach and found out what PST's are.
#sources: https://chess.stackexchange.com/questions/21998/in-chess-engines-a-i-how-are-the-values-for-piece-square-tables-for-simplified, https://rustic-chess.org/evaluation/psqt.html
# the idea is essentially create spots on the board that are to be prefered by creating values on specfic spots for specific pieces. It ended up working really well.
#based all these values off of the image from the first link
PawnPST = [
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    [5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0],
    [1.0, 1.0, 2.0, 3.0, 3.0, 2.0, 1.0, 1.0],
    [0.5, 0.5, 1.0, 2.5, 2.5, 1.0, 0.5, 0.5],
    [0.0, 0.0, 0.0, 2.0, 2.0, 0.0, 0.0, 0.0],
    [0.5, -0.5, -1.0, 0.0, 0.0, -1.0, -0.5, 0.5],
    [0.5, 1.0, 1.0,-2.0,-2.0, 1.0, 1.0,  0.5],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
]

KnightPST = [
    [-5.0,-4.0,-3.0,-3.0,-3.0,-3.0,-4.0,-5.0],
    [-4.0,-2.0, 0.0, 0.0, 0.0, 0.0,-2.0,-4.0],
    [-3.0, 0.0, 1.0, 1.5, 1.5, 1.0, 0.0,-3.0],
    [-3.0, 0.5, 1.5, 2.0, 2.0, 1.5, 0.5,-3.0],
    [-3.0, 0.0, 1.5, 2.0, 2.0, 1.5, 0.0,-3.0],
    [-3.0, 0.5, 1.0, 1.5, 1.5, 1.0, 0.5,-3.0],
    [-4.0,-2.0, 0.0, 0.5, 0.5, 0.0,-2.0,-4.0],
    [-5.0,-4.0,-3.0,-3.0,-3.0,-3.0,-4.0,-5.0]
]

BishopPST = [
    [-2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-2.0],
    [-1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,-1.0],
    [-1.0, 0.0, 0.5, 1.0, 1.0, 0.5, 0.0,-1.0],
    [-1.0, 0.5, 0.5, 1.0, 1.0, 0.5, 0.5,-1.0],
    [-1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0,-1.0],
    [-1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,-1.0],
    [-1.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.5,-1.0],
    [-2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-2.0]
]

RookPST = [
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    [0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5],
    [-0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,-0.5],
    [-0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,-0.5],
    [-0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,-0.5],
    [-0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,-0.5],
    [-0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,-0.5],
    [0.0, 0.0, 0.0, 0.5, 0.5, 0.0, 0.0, 0.0]
]

QueenPST = [
    [-2.0,-1.0,-1.0,-0.5,-0.5,-1.0,-1.0,-2.0],
    [-1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,-1.0],
    [-1.0, 0.0, 0.5, 0.5, 0.5, 0.5, 0.0,-1.0],
    [-0.5, 0.0, 0.5, 0.5, 0.5, 0.5, 0.0,-0.5],
    [ 0.0, 0.0, 0.5, 0.5, 0.5, 0.5, 0.0,-0.5],
    [-1.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.0,-1.0],
    [-1.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0,-1.0],
    [-2.0,-1.0,-1.0,-0.5,-0.5,-1.0,-1.0,-2.0]
]

KingPST = [
    [-3.0,-4.0,-4.0,-5.0,-5.0,-5.0,-4.0,-3.0],
    [-3.0,-4.0,-1.0, 0.0, 0.0,-1.0,-2.0,-3.0],
    [-3.0,-4.0, 2.0, 3.0, 3.0, 2.0,-1.0,-3.0],
    [-3.0,-4.0, 3.0, 4.0, 4.0, 3.0,-1.0,-3.0],
    [-3.0,-3.0, 3.0, 4.0, 4.0, 3.0,-1.0,-3.0],
    [-3.0,-1.0, 2.0, 3.0, 3.0, 2.0,-1.0,-3.0],
    [-3.0,-3.0, 0.0, 0.0, 0.0, 0.0,-3.0,-3.0],
    [-5.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-5.0]
]






In [9]:
#this is used to actually implement the values from the PST's
def GetPSTValue(PstPiece, PSTRow, PSTCol):
    if PstPiece == 'P':
        return PawnPST[PSTRow][PSTCol]
    elif PstPiece == 'p':
        return -PawnPST[7-PSTRow][PSTCol]
    elif PstPiece == 'T':
        return KnightPST[PSTRow][PSTCol]
    elif PstPiece == 't':
        return -KnightPST[7-PSTRow][PSTCol]
    elif PstPiece == 'B':
        return BishopPST[PSTRow][PSTCol]
    elif PstPiece == 'b':
        return -BishopPST[7-PSTRow][PSTCol]
    elif PstPiece == 'R':
        return RookPST[PSTRow][PSTCol]
    elif PstPiece == 'r':
        return -RookPST[7-PSTRow][PSTCol]
    elif PstPiece == 'Q':
        return QueenPST[PSTRow][PSTCol]
    elif PstPiece == 'q':
        return -QueenPST[7-PSTRow][PSTCol]
    elif PstPiece == 'K':
        return KingPST[PSTRow][PSTCol]
    elif PstPiece == 'k':
        return -KingPST[7-PSTRow][PSTCol]
    else:
        return 0

# this function will calculate the score on the board, if a move is performed
# give score for each of piece and calculate the score for the chess board
#combined values for each individual piece AND the PST's values. This allowed a way better just AI decision making in general.
#where I got the values(weights) for pieces: https://www.researchgate.net/post/AI-based_chess_games_and_weights_of_pieces#:~:text=In%20a%20computer%20chess%20game,%2D10%5E10%20or%20more.
def evl(EvlBoard):
    PieceVals = {'P': 1, 'T': 3, 'B': 3, 'R': 5, 'Q': 9, 'K': 100, 'p': -1, 't': -3, 'b': -3, 'r': -5, 'q': -9, 'k': -100}
    Eval = sum([PieceVals.get(PieceEvl, 0) for RowEvl in EvlBoard for PieceEvl in RowEvl])

    # bring in the PSTs
    for RowEvl in range(8):
        for ColEvl in range(8):
            PieceEvl = EvlBoard[RowEvl][ColEvl]
            if PieceEvl.isupper():
                Eval += GetPSTValue(PieceEvl, RowEvl, ColEvl)
            elif PieceEvl.islower():
                Eval -= GetPSTValue(PieceEvl, RowEvl, ColEvl)

    # makes stronk pieces like knights and bishops move more.
    for ColEvl in [1, 6]:  # knight columns
        if EvlBoard[0][ColEvl] == 'T':
            Eval -= 0.5
        if EvlBoard[7][ColEvl] == 't':
            Eval += 0.5
    for ColEvl in [2, 5]:  # bishop columns
        if EvlBoard[0][ColEvl] == 'B':
            Eval -= 0.5
        if EvlBoard[7][ColEvl] == 'b':
            Eval += 0.5
    # try to avoid doubled pawns
    for ColEvl in range(8):
        WhitePawns = sum(1 for RowEvl in range(8) if EvlBoard[RowEvl][ColEvl] == 'P')
        if WhitePawns > 1:
            Eval -= 0.5 * (WhitePawns - 1)
        BlackPawns = sum(1 for RowEvl in range(8) if EvlBoard[RowEvl][ColEvl] == 'p')
        if BlackPawns > 1:
            Eval += 0.5 * (BlackPawns - 1)
    # more score for center control
    for RowEvl in [3, 4]:
        for ColEvl in [3, 4]:
            if EvlBoard[RowEvl][ColEvl].isupper():
                Eval += 0.5
            elif EvlBoard[RowEvl][ColEvl].islower():
                Eval -= 0.5

    return Eval




def GetMinMaxMove(MMBoard, CurrentPlayerMM):
    # find the opposing player

    BestMoveMM = None
    BestValMM = float('-inf')
    OppostionMM = "black" if CurrentPlayerMM == "white" else "white"

    # find peices of all with legal moves of the current player
    Pieces4MM = GetPiecesWithLegalMoves(MMBoard, CurrentPlayerMM)
    for PieceMM in Pieces4MM:
        FromRowMM, FromColMM = PieceMM
        Moves4MM = GetListOfLegalMoves(MMBoard, CurrentPlayerMM, FromRowMM, FromColMM)
        # Test each legal move for the current piece
        for MoveMM in Moves4MM:
            # Create a copy of the board and apply the move
            CopyBoardMM = [row[:] for row in MMBoard]
            CopyBoardMM = MovePiece(CopyBoardMM, FromRowMM, FromColMM, *MoveMM)
            # Assess the worst-case scenario after making the move by evaluating the opposition's best response
            WorstCaseMM = float('inf')
            OPpiecesMM = GetPiecesWithLegalMoves(CopyBoardMM, OppostionMM)

            # check out opponetns pieces
            for OPpiece in OPpiecesMM:
                OPRow, OPCol = OPpiece
                OPMovesMM = GetListOfLegalMoves(CopyBoardMM, OppostionMM, OPRow, OPCol)
                # try out every legal moves of other piece
                for OPNMove in OPMovesMM:
                    OPCopyBoardMM = [row[:] for row in CopyBoardMM]
                    OPCopyBoardMM = MovePiece(OPCopyBoardMM, OPRow, OPCol, *OPNMove)
                    # Evaluate the board after the opponent's move
                    CurrentValue = evl(OPCopyBoardMM)
                    WorstCaseMM = min(WorstCaseMM, CurrentValue)
            # Ifcurrent worst case is better than previous updateit
            if WorstCaseMM > BestValMM:
                BestValMM = WorstCaseMM
                BestMoveMM = (PieceMM, MoveMM)

    # If best move was found return it
    if BestMoveMM:
        return BestMoveMM

    # If no best move found, return a random legal move to keep the game moving
    RandPieceMM = random.choice(Pieces4MM)
    FromRowMM, FromColMM = RandPieceMM
    RandMoveMM = random.choice(GetListOfLegalMoves(MMBoard, CurrentPlayerMM, FromRowMM, FromColMM))

    return RandPieceMM, RandMoveMM





In [ ]:
#this fucntion was used for testing the MinMax functionality.
#just print sout the game user can basically play against the MinMax AI
#SCRIPT DOES NOT HAVE ERRORS IF IT SAYS IT DOES IT"S BECAUSE I DID NTO ADD A Exit GAEM FUNCTIONALITY and only way to end a game early is by pressing the play button again and it creates an error try it, it works
def MMTEST():

    chessboard = ChessBoardSetup()
    CurrentPlayerMM = "white"

    while True:
        # Draw the current state of the board
        DrawBoard(chessboard)
        print(f"{CurrentPlayerMM.capitalize()}'s turn.")
        # Get the pieces that have legal moves for the active player
        PiecesWithLegalmovesMM = GetPiecesWithLegalMoves(chessboard, CurrentPlayerMM)

        # Check if player is in Checkmate
        if IsCheckmate(chessboard, CurrentPlayerMM):
            print(f"{CurrentPlayerMM.capitalize()} is in checkmate!")
            break
        # Check if player is in Check
        if IsInCheck(chessboard, CurrentPlayerMM):
            print(f"{CurrentPlayerMM.capitalize()} is in check!")

        # Check for stalemate
        elif not PiecesWithLegalmovesMM:
            print(f"{CurrentPlayerMM.capitalize()} is in stalemate!")
            break
        if CurrentPlayerMM == "black":
            # AI's turn
            FromPieceMM, ToPieceMM = GetMinMaxMove(chessboard, "black")
            FromRowMM, FromColMM = FromPieceMM
            ToRowMM, ToColMM = ToPieceMM
        else:
            # Real player's turn
            FromSquareMM, ToSquareMM = GetInput()
            # Convert Coords to row and col
            FromRowMM, FromColMM = CoordConverter(FromSquareMM)
            ToRowMM, ToColMM = CoordConverter(ToSquareMM)

        # Check if there's a piece on the starting square
        PieceToBeMovedMM = chessboard[FromRowMM][FromColMM]
        if not PieceToBeMovedMM:
            print("No piece at that location try again.")
            continue
        # Check if the piece belongs to the active player
        if (PieceToBeMovedMM.isupper() and CurrentPlayerMM == "black") or (PieceToBeMovedMM.islower() and CurrentPlayerMM == "white"):
            print("You can only move your pieces try again.")
            continue

        # Check move legality
        if IsMoveLegal(chessboard, FromRowMM, FromColMM, ToRowMM, ToColMM):
            # Make a copy of the board and try out the copy
            CopyBoardMM = [row[:] for row in chessboard]
            CopyBoardMM = MovePiece(CopyBoardMM, FromRowMM, FromColMM, ToRowMM, ToColMM)

            # Check if the move would put the player in check
            if IsInCheck(CopyBoardMM, CurrentPlayerMM):
                print("This move puts you in check try another move.")
                continue

            # If the move is valid, then update the actual board
            chessboard = CopyBoardMM
            # Switch active player
            if CurrentPlayerMM == "white":
                CurrentPlayerMM = "black"
            else:
                CurrentPlayerMM = "white"
        else:
            print("Illegal move try again.")

MMTEST()



In [11]:
def FINAL(N=75):  # N =75 I felt thsi was liek agood worst case average to nto take too long.
    chessboardFinal = ChessBoardSetup()
    CurrentPlayerFinal = "white"
    Turns = 0

    while not IsCheckmate(chessboardFinal, CurrentPlayerFinal) and Turns < N:
        clear_output(wait=True)  # clear previous board
        if CurrentPlayerFinal == "white":
            # MinMax AI = White Pieces
            (FromPieceFinal, MoveFinal) = GetMinMaxMove(chessboardFinal, CurrentPlayerFinal)
            chessboardFinal = MovePiece(chessboardFinal, FromPieceFinal[0], FromPieceFinal[1], MoveFinal[0], MoveFinal[1])
        else:
            # Random AI = Black Pieces
            (FromPieceFinal, MoveFinal) = GetRandomMove(chessboardFinal, CurrentPlayerFinal)
            chessboardFinal = MovePiece(chessboardFinal, FromPieceFinal[0], FromPieceFinal[1], MoveFinal[0], MoveFinal[1])
        # Swap player for next turn
        if CurrentPlayerFinal == "white":
            CurrentPlayerFinal = "black"
        else:
            CurrentPlayerFinal = "white"
        # write code to take turns and move the pieces
        Turns += 1
        print("Turn", Turns, ":")
        DrawBoard(chessboardFinal)
        time.sleep(0.5)
    # check and print - Stalemate or Checkmate
    if IsCheckmate(chessboardFinal, CurrentPlayerFinal):
        if CurrentPlayerFinal == "white":
            print("Black wins checkmate")
        else:
            print("White wins checkmate")
    else:
        print("Game is a stakemate after", N, "turns.")

FINAL()




Turn 23 :
r . . q k b . .
. b p . t p . r
. . . . . T p t
p p . . . . . p
. . . P P . . .
. . T K B Q . .
P P P . B P P P
R . . . . . . R
White wins checkmate
